In [ ]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory


load_dotenv()

# 1) 配置 GitHub Models（OpenAI 兼容）
llm = ChatOpenAI(
    model=os.getenv("GITHUB_MODEL", "gpt-4o-mini"),
    api_key=os.getenv("GITHUB_TOKEN"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.7,
    
)

# 2) Prompt：使用 MessagesPlaceholder 占位历史消息
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个友好、简洁的中文助理。请尽量简洁回答。"),
        MessagesPlaceholder(variable_name="history"),   # 历史对话将注入这里
        ("human", "{input}"),
    ]
)

# 3) 组合成可运行链
chain = prompt | llm

# 4) 用 RunnableWithMessageHistory 包装——官方推荐的“新式记忆”用法
#    需要提供一个函数：给 session_id 返回对应的 ChatMessageHistory
store = {}  # 简单内存存储；实际可替换为 Redis/DB 等

def get_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

conversational_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",      # 对应 prompt 里的 {input}
    history_messages_key="history",  # 对应 MessagesPlaceholder 的名字
)

# 5) 进行多轮对话
session_id = "user-001"

def ask(user_text: str):
    resp = conversational_chain.invoke(
        {"input": user_text},
        config={"configurable": {"session_id": session_id}},
    )
    # resp 是 AIMessage
    return resp.content

print(ask("你好，我叫小王。"))
print(ask("我刚才说我叫什么？"))
print(ask("以后请用一句话回答。我的职业是什么（如果不知道就说不知道）？"))

# 你也可以随时查看历史：
hist = get_history(session_id).messages
for m in hist:
    role = "人类" if isinstance(m, HumanMessage) else "助手" if isinstance(m, AIMessage) else "系统"
    print(f"[{role}] {m.content}")
